# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

#prevent from going idle
from workspace_utils import active_session

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r"http\S*", " ", text)
        
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r"\$\S*"," ",text)
        
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r"@\S*"," ",text)
    
    # Replace everything not a letter with a space
    text = re.sub(r"[\W0-9_]", " ", text)
    
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()
    

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w) for w in tokens if len(w)>1]
    
    return tokens


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement

tokenized = [preprocess(m) for m in messages]


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = Counter([w for t in tokenized for w in t])


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [23]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
word_count = len([word for sentence in tokenized for word in sentence])
print('Word Count:',word_count)
freqs = {key: val/word_count for key, val in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 5/word_count

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 100000/word_count

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [w for w in bow.keys() if freqs[w] > high_cutoff]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print('Most Common: ', K_most_common)
print('# Filtered Words:', len(filtered_words)) 

Word Count: 15529983
Most Common:  ['at', 'for', 'of', 'is', 'on', 'the', 'to', 'in', 'and', 'this', 'it', 'will', 'up', 'are']
# Filtered Words: 24827


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [24]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement



# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {fw:index for index, fw in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {index: fw for index, fw in enumerate(filtered_words)}
# tokenized with the words not in `filtered_words` removed.
words_not_in_filtered_words = [word for word in freqs if (freqs[word] <= low_cutoff or word in K_most_common)]
set_filtered_words = set(filtered_words)
filtered = [[w for w in t if w in set_filtered_words] for t in tokenized ]



### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [25]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [26]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19527652564030418

Finally let's convert our tokens into integer ids which we can pass to the network.

In [27]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']
print(len(token_ids))
print(len(sentiments))

1030682
1030682


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [28]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers, bias=False, batch_first=False, dropout=self.dropout)
        self.dense = nn.Linear(self.lstm_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=2)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        from torch.autograd import Variable
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        hidden = Variable(next(self.parameters()).data.new(self.lstm_layers, batch_size, self.lstm_size), requires_grad=False)
        cell = Variable(next(self.parameters()).data.new(self.lstm_layers, batch_size, self.lstm_size), requires_grad=False)
        return hidden.zero_(), cell.zero_()
        
        
    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """        
        # TODO Implement 
        x = self.embedding(nn_input)
        x, hidden = self.lstm(x, hidden_state)
        x = self.dense(x)
        #print("output_dense", x)
        x = self.softmax(x)
        #print("output_softmax", x)
        x= x[-1, :,:]
        return x, hidden

### View Model

In [29]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.9596, -1.4504, -1.4571, -1.7982, -1.4867],
        [-1.9572, -1.4817, -1.4495, -1.7959, -1.4661],
        [-1.9308, -1.4714, -1.4570, -1.7824, -1.4952],
        [-1.9735, -1.4647, -1.4435, -1.8065, -1.4716]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [30]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [32]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
num_messages = len(token_ids)
num_train = int(num_messages * 0.75)
num_valid = int(num_messages * 0.25)
train_features = token_ids[:num_train]
valid_features = token_ids[num_train:]
train_labels = sentiments[:num_train]
valid_labels = sentiments[num_train:]


In [33]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(24828, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, bias=False, dropout=0.2)
  (dense): Linear(in_features=512, out_features=5, bias=True)
  (softmax): LogSoftmax()
)

In [60]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
#to remove
import numpy as np

epochs = 5 #pass
batch_size = 500 #pass
learning_rate = 0.001 #pass 

print_every = 100 # set to 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

with active_session():
    for epoch in range(epochs):
        print('Starting epoch {}'.format(epoch + 1))

        steps = 0
        for text_batch, labels in dataloader(
                train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
            steps += 1
            hidden = model.init_hidden(labels.shape[0])#pass
            #print("text_batch", text_batch)
            #print("label", labels)
            # Set Device
            text_batch, labels = text_batch.to(device), labels.to(device)
            for each in hidden:
                each.to(device)

            # TODO Implement: Train Model
            model.zero_grad()
            output, hidden = model(text_batch, hidden)
            #print("output", output)
            loss = criterion(output.squeeze(), labels.long())
            loss.backward()

            optimizer.step()

            if steps % print_every == 0:
                model.eval()

                # TODO Implement: Print metrics
                val_losses = []
                corrects = []
                for v_text_batch, v_labels in dataloader(
                    valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                    hidden = model.init_hidden(v_labels.shape[0])
                    v_text_batch, v_labels = v_text_batch.to(device), v_labels.to(device)
                    for each in hidden:
                        each.to(device)
                    v_output, hidden = model(v_text_batch, hidden)
                    v_loss = criterion(v_output.squeeze(), v_labels.long())
                    val_losses.append(v_loss.item())
                    v_output2 = torch.argmax(v_output,1)
                    corrects.append(((v_output2 == v_labels).float().sum()/v_labels.shape[0]))
                    
                model.train()
                print("Epoch: {}/{}...".format(epoch, epochs),
                      "Step: {}...".format(steps),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)),
                      "Acc: {:.3f}...".format(np.mean(corrects)))

Starting epoch 1
Epoch: 0/5... Step: 100... Loss: 0.697167... Val Loss: 0.742627 Acc: 0.715...
Epoch: 0/5... Step: 200... Loss: 0.740467... Val Loss: 0.748781 Acc: 0.717...
Epoch: 0/5... Step: 300... Loss: 0.715104... Val Loss: 0.739115 Acc: 0.715...
Epoch: 0/5... Step: 400... Loss: 0.665616... Val Loss: 0.736666 Acc: 0.719...
Epoch: 0/5... Step: 500... Loss: 0.716132... Val Loss: 0.733036 Acc: 0.721...
Epoch: 0/5... Step: 600... Loss: 0.694753... Val Loss: 0.734182 Acc: 0.719...
Epoch: 0/5... Step: 700... Loss: 0.708477... Val Loss: 0.732346 Acc: 0.720...
Epoch: 0/5... Step: 800... Loss: 0.676088... Val Loss: 0.727082 Acc: 0.721...
Epoch: 0/5... Step: 900... Loss: 0.633618... Val Loss: 0.722599 Acc: 0.724...
Epoch: 0/5... Step: 1000... Loss: 0.628096... Val Loss: 0.721324 Acc: 0.724...
Epoch: 0/5... Step: 1100... Loss: 0.655149... Val Loss: 0.721861 Acc: 0.724...
Epoch: 0/5... Step: 1200... Loss: 0.619623... Val Loss: 0.718580 Acc: 0.724...
Epoch: 0/5... Step: 1300... Loss: 0.702024..

KeyboardInterrupt: 

In [56]:
#THIS CELL IS THERE ONLY FOR CHECK PURPOSES. WILL BE DELETED WHEN SUBMITTED
v_text_batch, v_labels = next(dataloader(
                    valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True))

print(v_labels)

hidden = model.init_hidden(v_labels.shape[0])
v_text_batch, v_labels = v_text_batch.to(device), v_labels.to(device)
for each in hidden:
    each.to(device)
v_output, hidden = model(v_text_batch, hidden)
print(v_output.shape)
print(v_output)
print(torch.argmax(v_output, 1))
print(torch.argmax(v_output, 1).shape)
v_output = torch.argmax(v_output, 1)
correct = (v_output == v_labels).float().sum()
print(correct)
print(correct / v_labels.shape[0])

tensor([ 3,  0,  2,  2,  3,  4,  3,  2,  4,  4,  3,  1,  3,  0,
         1,  4,  2,  3,  4,  0,  0,  3,  4,  2,  4,  2,  3,  2,
         3,  1,  1,  0,  3,  4,  3,  3,  0,  1,  1,  1,  4,  3,
         1,  3,  4,  0,  0,  2,  1,  0,  4,  1,  3,  2,  4,  2,
         0,  4,  2,  2,  3,  3,  3,  0,  0,  0,  2,  1,  3,  4,
         4,  4,  4,  2,  2,  3,  0,  3,  3,  2,  4,  0,  0,  2,
         3,  3,  4,  1,  3,  0,  3,  1,  3,  4,  2,  1,  0,  3,
         3,  3,  1,  4,  4,  4,  3,  4,  3,  3,  4,  1,  3,  1,
         3,  0,  3,  2,  1,  2,  1,  4,  3,  2,  4,  4,  2,  0,
         1,  0,  1,  0,  3,  3,  4,  0,  0,  0,  1,  1,  4,  2,
         4,  4,  1,  4,  0,  3,  3,  0,  0,  1,  3,  1,  4,  4,
         2,  4,  1,  3,  2,  1,  2,  1,  3,  0,  2,  3,  2,  3,
         0,  4,  4,  3,  3,  4,  4,  3,  4,  1,  3,  2,  0,  0,
         1,  2,  3,  1,  2,  1,  4,  4,  1,  1,  2,  0,  2,  2,
         0,  3,  2,  1,  2,  1,  1,  1,  4,  3,  0,  2,  4,  0,
         2,  4,  0,  1,  4,  1,  1,  1, 

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [74]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [t for t in tokens if t in set_filtered_words]
    
    # Convert words to ids
    tokens = [vocab[t] for t in tokens]
        
    # Adding a batch dimension
    text_input = [tokens]
    text_input = torch.LongTensor(text_input)
    # Get the NN output
    hidden = model.init_hidden(text_input.shape[1])
    
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [75]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
res = predict(text, model, vocab)
res

tensor([[ 5.4967e-05,  1.3315e-03,  5.4574e-01,  4.5240e-01,  4.7792e-04],
        [ 2.8215e-06,  5.5842e-07,  9.9591e-01,  4.0888e-03,  9.3803e-08],
        [ 8.7048e-05,  9.9707e-01,  2.5986e-03,  2.3390e-04,  9.0781e-06],
        [ 2.2801e-01,  8.2582e-03,  6.8136e-01,  8.2374e-02,  2.8405e-06],
        [ 2.7981e-04,  4.4615e-04,  9.9656e-01,  9.6300e-04,  1.7476e-03],
        [ 1.5506e-10,  4.2665e-03,  1.8175e-04,  9.0480e-01,  9.0750e-02]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**
Regarding this single prediction the output is 1 in a range from -2 to 2, from very negative to very positive. So it is positive. The uncertainty is low, then top value for 1 is 0.90480, second top value for label 2 is 0.09075 and next label -1 is 0.00426, and so on. The secondly scored label lies 10 times lower in score than the first one. So that's quite certain. However the more epochs we train and reduce the error, which after 25 epochs was pushed down to below 0.2, this result would improve by still enlarging the difference in magnite between the first and second scored labels.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [63]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [64]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [65]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [69]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.0062,  0.0127,  0.4626,  0.0042,  0.5144],
         [ 0.0006,  0.0309,  0.1759,  0.7754,  0.0172]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.